### Data collection

In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import pandas as pd
pd.options.display.max_columns = None # See all columns
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

##### Let's add in strength of schedule for each player

In [55]:
strength = pd.read_csv("strength2020.csv", header=0)

# Invert matrix for merging purposes
inverted_strength = pd.melt(strength, id_vars=['Team'], value_vars=['QB', 'RB', 'WR', 'TE'], var_name='Pos', value_name='Rank')

# Make sure the Team columns are the same for merging
inverted_strength = inverted_strength.rename(columns={'Team': 'Tm'})

# Lets capitalize each first letter of team
inverted_strength['Tm'] = inverted_strength['Tm'].str.capitalize()

inverted_strength

,Tm,Pos,Rank
0,Ari,QB,20
1,Atl,QB,23
2,Bal,QB,7
3,Buf,QB,31
4,Car,QB,2
...,...,...,...
123,Sf,TE,3
124,Sea,TE,14
125,Tb,TE,22
126,Ten,TE,25


##### Now, let's update our dataset to include these ranks  

To perform this merge, we will need to make our original dataset team values all uppercase

In [56]:
merged['Tm'] = merged['Tm'].str.capitalize()

# Lets perform the merge, fill FA as average strength
updated_df = pd.merge(merged, inverted_strength, on=['Tm', 'Pos'], how='left').fillna(16)

##### We now have our strength of schedule included in our dataset:

##### Now, lets divide all game-statistics columns by total games played so we receive the per-game stats. This will help factor out injuries and instead focus on production.

In [57]:
updated_df.iloc[:, 4:-2] = updated_df.iloc[:, 4:-2].div(updated_df['GP'], axis=0)